# Benchmarks experiments notebook
This notebook is an accompanying notebook to the main script <code>experiments_scripts.py</code>. 
It can be used to tune the hyper-parameters ranges and for quick testing. To run this notebook and the script, first install <code>darts</code> and the [<code>ray</code> library](https://docs.ray.io/en/latest/ray-overview/installation.html). Install [<code>optuna</code> library](https://optuna.readthedocs.io/en/stable/installation.html) to use Optuna search algorithms.  

# read data, setup directories, transform 

In [ ]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline 


from sklearn.preprocessing import MaxAbsScaler
from darts.dataprocessing.transformers import Scaler
from ray import tune

from datetime import datetime
import random
import os
from tqdm import tqdm
from darts.utils.utils import series2seq
from darts.dataprocessing.pipeline import Pipeline
from darts.metrics import mse, mae, smape, rmse, mape, mase
import torch
from pytorch_lightning.callbacks import Callback, EarlyStopping
from darts import TimeSeries
import numpy as np
import pickle
import json
import ray
from ray import tune, air
from ray.tune.search.optuna import OptunaSearch
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from darts.models.forecasting.regression_model import RegressionModel
from darts.models.forecasting.torch_forecasting_model import (MixedCovariatesTorchModel, PastCovariatesTorchModel,
                                                              FutureCovariatesTorchModel , TorchForecastingModel)
from ray.tune.schedulers import ASHAScheduler
from ray.air import session
import matplotlib.pyplot as plt
import optuna
from csv import DictWriter
from darts.utils import missing_values
from darts.utils.statistics import plot_pacf

# data and models
from darts.datasets import ETTh1Dataset, ElectricityDataset
from darts.models import TCNModel, DLinearModel, NLinearModel, NHiTSModel, LightGBMModel, LinearRegressionModel
from builders import MODEL_BUILDERS


In [ ]:
IN_MIN = 5
IN_MAX = 30 

def _params_LinearRegression(trial):

    lags = trial.suggest_int("lags", IN_MIN * PERIOD_UNIT, IN_MAX * PERIOD_UNIT)
    out_len = trial.suggest_int("out_len", 1, lags - PERIOD_UNIT)

    add_encoders = trial.suggest_categorical("add_encoders", [False, True])

    return None

def _params_LGBMModel(trial):

    lags = trial.suggest_int("lags", IN_MIN * PERIOD_UNIT, IN_MAX * PERIOD_UNIT)
    out_len = trial.suggest_int("out_len", 1, lags - PERIOD_UNIT)

    boosting = trial.suggest_categorical("boosting", ["gbdt", "dart"])
    num_leaves = trial.suggest_int("num_leaves", 2, 50)
    max_bin = trial.suggest_int("max_bin", 100, 500)
    learning_rate = trial.suggest_float("learning_rate", 1e-8, 1e-1, log=True)
    num_iterations = trial.suggest_int("num_iterations", 50, 500)
    add_encoders = trial.suggest_categorical("add_encoders", [False, True])

    return None

def _params_NHITS(trial):
    
    in_len = trial.suggest_int("in_len", IN_MIN * PERIOD_UNIT, IN_MAX * PERIOD_UNIT) 
    out_len = trial.suggest_int("out_len", 1, in_len - PERIOD_UNIT)
    num_stacks = trial.suggest_int("num_stacks", 2,5)
    num_blocks = trial.suggest_int("num_blocks", 1,3)
    num_layers = trial.suggest_int("num_layers", 2,5)
    activation = trial.suggest_categorical("activation", 
                                          ['ReLU','RReLU', 'PReLU', 'Softplus', 'Tanh', 'SELU', 'LeakyReLU', 'Sigmoid'])
    MaxPool1d = trial.suggest_categorical("MaxPool1d", [False, True])
    dropout = trial.suggest_float("dropout", 0.0, 0.4)
    
    lr = trial.suggest_float("lr", 5e-5, 1e-3, log=True)
    add_encoders = trial.suggest_categorical("add_encoders", [False, True])
    
    constants = {"layer_widths": 512, "pooling_kernel_sizes": None,
                "n_freq_downsample" : None, }

    return constants

def _params_NLINEAR(trial):
    
    in_len = trial.suggest_int("in_len", IN_MIN * PERIOD_UNIT, IN_MAX * PERIOD_UNIT) 
    out_len = trial.suggest_int("out_len", 1, in_len - PERIOD_UNIT)
    shared_weights = trial.suggest_categorical("shared_weights", [False, True])
    const_init = trial.suggest_categorical("const_init", [False, True])
    normalize = trial.suggest_categorical("normalize", [False, True])
    
    lr = trial.suggest_float("lr", 5e-5, 1e-3, log=True)
    add_encoders = trial.suggest_categorical("add_encoders", [False, True])

    return None

def _params_DLINEAR(trial):

    in_len = trial.suggest_int("in_len", IN_MIN * PERIOD_UNIT, IN_MAX * PERIOD_UNIT) 
    out_len = trial.suggest_int("out_len", 1, in_len - PERIOD_UNIT)
    kernel_size = trial.suggest_int("kernel_size", 5, 25)
    shared_weights = trial.suggest_categorical("shared_weights", [False, True])
    const_init = trial.suggest_categorical("const_init", [False, True])
    
    lr = trial.suggest_float("lr", 5e-5, 1e-3, log=True)
    add_encoders = trial.suggest_categorical("add_encoders", [False, True])

    return None


def _params_TCNMODEL(trial):
    
    in_len = trial.suggest_int("in_len", IN_MIN * PERIOD_UNIT, IN_MAX * PERIOD_UNIT) 
    out_len = trial.suggest_int("out_len", 1, in_len - PERIOD_UNIT)
    kernel_size = trial.suggest_int("kernel_size", 5, 25)
    num_filters = trial.suggest_int("num_filters", 5, 25)
    weight_norm = trial.suggest_categorical("weight_norm", [False, True])
    dilation_base = trial.suggest_int("dilation_base", 2, 4)
    dropout = trial.suggest_float("dropout", 0.0, 0.4)
    
    lr = trial.suggest_float("lr", 5e-5, 1e-3, log=True)
    add_encoders = trial.suggest_categorical("add_encoders", [False, True])

    return None


params_generators = {
    TCNModel.__name__: _params_TCNMODEL,
    DLinearModel.__name__:_params_DLINEAR,
    NLinearModel.__name__:_params_NLINEAR,
    NHiTSModel.__name__:_params_NHITS, 
    LightGBMModel.__name__:_params_LGBMModel, 
    LinearRegressionModel.__name__:_params_LinearRegression
}


## configure experiment

In [ ]:
dataset = ETTh1Dataset
model_cl = DLinearModel#LinearRegressionModel#LightGBMModel#DLinearModel#TCNModel #NHiTSModel#
random_seed = 42 

# data
PERIOD_UNIT = 24 
subset_size = int(365*1.5) * PERIOD_UNIT
split = 0.7
load_as_multivariate = False 
encoders_past = {"datetime_attribute": {"past": ["month", "week", "hour","dayofweek"]},
                "cyclic": {"past": ["month", "week", "hour", "dayofweek"]}} 
encoders_future = {"datetime_attribute": {"future": ["month", "week", "hour","dayofweek"]},
                    "cyclic": {"future": ["month", "week", "hour", "dayofweek"]}}

encoders = encoders_future if issubclass(model_cl, (MixedCovariatesTorchModel, FutureCovariatesTorchModel,RegressionModel))\
            else encoders_past

# model training
fixed_params={
    "BATCH_SIZE" : 1024,
    "MAX_N_EPOCHS": 100,
    "NR_EPOCHS_VAL_PERIOD": 1,
    "MAX_SAMPLES_PER_TS": 1000, 
    "RANDOM_STATE": random_seed
}

train_with_metric = True # whether optimize models based on a metric or based on val_loss
eval_metric = smape
time_budget = 300 # in seconds


## setup logging directory

In [ ]:
#Fix random states
#https://pytorch.org/docs/stable/notes/randomness.html
random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)
torch.use_deterministic_algorithms(True)

exp_start_time = datetime.now()
exp_name = f"{model_cl.__name__}_{exp_start_time.strftime('%Y-%m-%d')}_pid{os.getpid()}_seed{random_seed}"

# create directories 
experiment_root = os.path.join(os.getcwd(), f"benchmark_experiments/{dataset.__name__}")
experiment_dir = os.path.join(os.getcwd(), f"{experiment_root}/{exp_name}")
print(experiment_dir)
os.makedirs(experiment_dir, exist_ok=True)

## read data and split

In [ ]:

#read data 
if "multivariate" in dataset.__init__.__code__.co_varnames:
    data = dataset(multivariate=load_as_multivariate).load()
else:
    data = dataset().load()

data = series2seq(data)

data = [
    s[-subset_size:].astype(np.float32) for s in tqdm(data)
]


# split : train, validation , test (validation and test have same length)
all_splits = [list(s.split_after(split)) for s in data]
train_original = [split[0] for split in all_splits]
vals = [split[1] for split in all_splits]
vals = [list(s.split_after(0.5)) for s in vals]
val_original = [s[0] for s in vals]
test_original = [s[1] for s in vals]


train_len = len(train_original[0])
val_len = len(val_original[0])
test_len = len(test_original[0])
num_series = len(train_original)
n_components = train_original[0].n_components

print("number of series:", num_series)
print("number of components:", n_components)
print("training series length:", train_len)
print("validation series length:", val_len)
print("test series length:", test_len)

## Check missing values and transform data

In [ ]:
# check if missing values and fill
for i in range(num_series):
    missing_ratio = missing_values.missing_values_ratio(train_original[i])
    print(f"missing values ratio in training series {i} = {missing_ratio}")
    print("filling training missing values by interpolation")
    if missing_ratio > 0.0:
        missing_values.fill_missing_values(train_original[i])
    
    missing_ratio = missing_values.missing_values_ratio(val_original[i])
    print(f"missing values ratio in validation series {i} = {missing_ratio}")
    print("filling validation missing values by interpolation")
    if missing_ratio > 0.0:
        missing_values.fill_missing_values(val_original[i])
    
    missing_ratio = missing_values.missing_values_ratio(test_original[i])
    print(f"missing values ratio in test series {i} = {missing_ratio}")
    print("filling test missing values by interpolation")
    if missing_ratio > 0.0:
        missing_values.fill_missing_values(test_original[i])
    

In [ ]:
scaler = (
        Scaler(scaler=MaxAbsScaler())
        if issubclass(model_cl, TorchForecastingModel)
        else None
    )

if scaler is not None:
    train = scaler.fit_transform(train_original)
    val = scaler.transform(val_original)
    test = scaler.transform(test_original)
else:
    train = train_original
    val = val_original
    test = test_original

## plot some data and checks

In [ ]:
# pick series and components at random to plot 
max_series_to_plot = 5
max_comps_to_plot = 5


if n_components < max_comps_to_plot:
    max_comps_to_plot = n_components

comps_vec = np.random.randint(0, n_components, max_series_to_plot)
comp_names = train[0].columns.to_list()

if num_series < max_series_to_plot:
    max_series_to_plot = num_series

print("all components:", comp_names)
series_vec = np.random.randint(0, num_series, max_series_to_plot)

for idx in series_vec:
    for comp_id in comps_vec:
        comp_id = comp_names[comp_id]
        plt.figure(figsize=(15, 5))
        val[idx][comp_id].plot()
        plt.title(f"{dataset.__name__}_{comp_id}_series{idx}")
        plt.show()
        plt.close()
        
        plot_pacf(val[idx][comp_id], max_lag = IN_MAX * PERIOD_UNIT)


# setup optimiztation function and tuner

In [ ]:
# objective function 

def objective_val_loss(config, model_cl, encoders, fixed_params, train=train, val=val):

    metrics = {"metric":"val_loss"}

    tuner_callbacks = [TuneReportCallback(metrics, on="validation_end")]

    model = MODEL_BUILDERS[model_cl.__name__](**config, fixed_params= fixed_params, encoders = encoders, 
                                              callbacks=tuner_callbacks)

    # train the model
    if "val_series" in model.fit.__code__.co_varnames:
        model.fit(
            series=train,
            val_series=val,
            max_samples_per_ts=fixed_params["MAX_SAMPLES_PER_TS"])
    else:
        model.fit(
            series=train,
            max_samples_per_ts=fixed_params["MAX_SAMPLES_PER_TS"])

    

def objective_metric(config, model_cl, metric, encoders, fixed_params, train=train, val=val):
    
    model = MODEL_BUILDERS[model_cl.__name__](**config, encoders = encoders, fixed_params=fixed_params)

    # train the model
    if "val_series" in model.fit.__code__.co_varnames:
        model.fit(
            series=train,
            val_series=val,
            max_samples_per_ts=fixed_params["MAX_SAMPLES_PER_TS"])
    else:
        model.fit(
            series=train,
            max_samples_per_ts=fixed_params["MAX_SAMPLES_PER_TS"])

    # DL Models : use best model for subsequent evaluation
    if isinstance(model, TorchForecastingModel):
        model = model_cl.load_from_checkpoint(model_cl.__name__, work_dir = os.getcwd(), best = True)

    preds = model.predict(series=train, n=val_len)

    
    if metric.__name__ == "mase":
        metric_evals = metric(val, preds, train, n_jobs=-1, verbose=True)
    else:
        metric_evals = metric(val, preds, n_jobs=-1, verbose=True)

    metric_evals_reduced = np.mean(metric_evals) if metric_evals != np.nan else float("inf")

    session.report({"metric":metric_evals_reduced})
    

    
objective_metric_with_params = tune.with_parameters(objective_metric, model_cl=model_cl, metric = eval_metric, 
                                            encoders = encoders, fixed_params=fixed_params, train=train, val=val)

objective_val_loss_with_params = tune.with_parameters(objective_val_loss, model_cl=model_cl, 
                                            encoders = encoders, fixed_params=fixed_params, train=train, val=val)

search_alg = OptunaSearch(
    space = params_generators[model_cl.__name__],
    metric= "metric",
    mode= "min",
)

tuner =  tune.Tuner(
            trainable=objective_metric_with_params if train_with_metric else objective_val_loss_with_params,
            tune_config = tune.TuneConfig(
                search_alg = search_alg,
                num_samples = -1,
                time_budget_s = time_budget,
            ),
            run_config = air.RunConfig(
                                       local_dir = experiment_dir,
                                       name = f"{model_cl.__name__}_tuner_{eval_metric.__name__}")
        )

    

# run hyperparameters tuner

In [ ]:
# run optimizer
tuner_results = tuner.fit()

In [ ]:
# get best results 
best_params = tuner_results.get_best_result(metric = "metric", mode = "min").config
print("best parameters:", best_params)

# train best model

In [ ]:

best_model = MODEL_BUILDERS[model_cl.__name__](**best_params, 
                                               encoders = encoders,
                                               fixed_params=fixed_params, work_dir=experiment_dir)

if isinstance(best_model, TorchForecastingModel):
    best_model.n_epochs = fixed_params["MAX_N_EPOCHS"] + 50

train_start_time= datetime.now()
# train the model
if "val_series" in best_model.fit.__code__.co_varnames:
    best_model.fit(
        series=train,
        val_series=val,
        max_samples_per_ts=fixed_params["MAX_SAMPLES_PER_TS"],
    )
else:
    best_model.fit(
        series=train,
        max_samples_per_ts=fixed_params["MAX_SAMPLES_PER_TS"],
    )
train_end_time = datetime.now()
training_time = (train_end_time - train_start_time).total_seconds()

# inference with best model

In [ ]:
inference_start_time = datetime.now()
test_predictions = best_model.predict(series = val, n = test_len)
inference_end_time = datetime.now()
inference_time = (inference_end_time - inference_start_time).total_seconds()

## best model performance

In [ ]:
if eval_metric.__name__ == "mase":
    metric_evals = eval_metric(test, test_predictions, train, n_jobs=-1, verbose=True)
else:
    metric_evals = eval_metric(test, test_predictions, n_jobs=-1, verbose=True)

metric_evals_mean = np.mean(metric_evals) if metric_evals != np.nan else float("inf")
metric_evals_std= np.std(metric_evals)
print(f"{eval_metric.__name__} mean = {metric_evals_mean}, std = {metric_evals_std}")

# plot forecasts

In [ ]:
# pick series and components at random to plot 
max_series_to_plot = 5
max_comps_to_plot = 5


if n_components < max_comps_to_plot:
    max_comps_to_plot = n_components

comps_vec = np.random.randint(0, n_components, max_series_to_plot)
comp_names = train[0].columns.to_list()

if num_series < max_series_to_plot:
    max_series_to_plot = num_series

print("all components:", comp_names)
series_vec = np.random.randint(0, num_series, max_series_to_plot)

for idx in series_vec:
    for comp_id in comps_vec:
        comp_id = comp_names[comp_id]
        plt.figure(figsize=(15, 5))
        val[idx][comp_id][-(val_len//4):].plot()
        test[idx][comp_id].plot(label='actual')
        test_predictions[idx][comp_id].plot(label='forecast')
        plt.title(f"{dataset.__name__}_{comp_id}_series{idx}")
        plt.show()
        plt.close()